In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import matplotlib.pyplot as plt
from scipy import stats
from scipy import cluster

In [2]:
n = 1000
beta01, beta11 = 5,-3
beta02, beta12 = 2, 4

#set up regression mixture
x1 = np.random.uniform(0, 10, size=400)
x2 = np.random.uniform(0, 10, size=600)

y1 = beta01 + beta11*x1 + np.random.normal(scale=5.0, size=400)
y2 = beta02 + beta12*x2 + np.random.normal(scale=4.0,size=600)

X = np.concatenate([x1, x2])
Y = np.concatenate([y1, y2])


#set up 2 component mixture
a1 = np.random.normal(2, 5, size=600)
a2 = np.random.normal(5, 3, size=400)
a = np.concatenate([a1,a2])

In [3]:
def e_step(y,x,params): 
    y, x = np.array(y), np.array(x)
    nobs, k = x.shape
    weights = []
    for param in params:

        sigma = param[-1]
        beta = np.tile(param[1:-1],nobs).reshape(nobs, k)
        mean = (beta*x).sum(axis=1)
        weights.append( stats.norm.pdf(y, loc=mean, scale=sigma)*param[0] )
        
    #update loop variables
    weights = np.array(weights).transpose()
    #denom = np.repeat( 1+np.exp(weights).sum(axis=1), len(params) ).reshape(nobs,len(params))
    denom = np.repeat(weights.sum(axis=1), len(params) ).reshape(nobs,len(params))
    weights = weights/denom
    return weights, np.log(denom[:,0])
        
    
def m_step(y,x,weights):
    y, x, weights = np.array(y), np.array(x), np.array(weights)
    nobs, k = x.shape
    params, se, err = [], [], 0

    for w in weights.transpose():
        
        lamb = w.mean()
        lamb_se = w.std()

        #beta
        w_mat = np.diag(w)
        xx_mat = np.linalg.inv( x.transpose().dot( w_mat).dot(x) )
        beta = xx_mat.dot(x.transpose().dot(w_mat)).dot(y)
        
        #sigma
        mu = np.tile(beta, nobs).reshape(nobs, k)*x
        weighted_err = w*(y - mu.sum(axis=1))**2
        sigma =  (weighted_err.sum()/w.sum())**.5

        #add component
        comp_param =np.concatenate(([lamb],beta,[sigma]))
        params.append(comp_param)

        #beta_se
        beta_se = (np.diagonal(xx_mat*sigma**2))**.5
        comp_se = np.concatenate(([lamb_se],beta_se))
        se.append(comp_se)

        #SSR
        err = err+weighted_err
    return np.array(params), np.array(se), 1-err.mean()/y.var()


def gen_weights(y,ncomp):
    c,labels = cluster.vq.kmeans2(y,ncomp)
    return np.array(pd.get_dummies(labels))


def estimate(y,x,ncomp):
    e = gen_weights(y,ncomp)
    m = None
    for i in range(20):
        m,se,r2 = m_step(y,x,e)
        e,ll = e_step(y,x,m)
    return m, se, r2, y, x, ncomp, ll


m, se, r2, y, x, ncomp, ll = estimate(Y, sm.add_constant(X), 1)

In [60]:
def write_table(fname, estimates, labels=('y',None)):
    
    #unpack relevant information
    params, se, r2, y, x, ncomp, ll = estimates
    nobs, k = x.shape
    ylabel, xlabel = labels
    
    #calc aic
    aic = 2*(params.shape[0]*params.shape[1]-2) - 2*ll.sum()
    aic = np.round(aic,1)
    
    if xlabel == None:
        xlabel =[]
        for i in range(k):
            xlabel.append('x%s'%i)
            
    assert (k == len(xlabel)) 
    
    f = open(fname, "w+")
    
    f.write(('\\begin{center}  \n'+
            '\\begin{tabular}{lclc} \n'+
            '\\toprule \n'+
            '\\textbf{Dep. Variable:} & %s & \\textbf{  R-squared: } &  %s \\\\ \n'%(ylabel, np.round(r2,3))  ))
    
    f.write(('\\textbf{No. Observations:} & %s & \\textbf{ AIC:} & %s \\\\ \n'%(nobs,aic)+
                                                                                    
            '\end{tabular} \n'))
    
    
    f.write('\n\\begin{tabular}{lcccc} \n')
    for comp in range(ncomp):
        f.write('\\toprule \n')
        f.write('\\textbf{History %s} & \\textbf{est} & \\textbf{std err} &'%(1+comp)+ 
                '\\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\ \n')
        f.write('\\bottomrule \\\\ \n')
        
        #isolate params
        comp_params = params[comp]
        comp_se = se[comp]
        comp_t = comp_params[:-1]/comp_se
        comp_p = 1 - stats.t.cdf(np.abs(comp_t),df=(nobs-k)) + stats.t.cdf(-np.abs(comp_t),df=(nobs-k))
        
        #round everything
        comp_params = np.round(comp_params,5)
        comp_se = np.round(comp_se,5)
        comp_t = np.round(comp_t,5)
        comp_p = np.round(comp_p,5)
        
        lamb = comp_params[0]
        #lamb_se, lamb_t, lamb_p = comp_params[0], comp_se[0], comp_t[0], comp_p[0]
        beta, beta_se, beta_t, beta_p = comp_params[1:-1], comp_se[1:], comp_t[1:], comp_p[1:]
        sigma = comp_params[-1]
        
        if ncomp > 1:
            f.write('\\textbf{lambda} & %s  & & & \\\\ \\\\ \n'%(lamb) )
        
        for i in range(k):
            f.write('\\textbf{%s} & %s & (%s) & %s & %s \\\\ \\\\ \n'%(xlabel[i],beta[i],beta_se[i],
                                                                             beta_t[i],beta_p[i]) )
        
        f.write('\\textbf{sigma} & %s &  & & \\\\ \\\\ \n'%(sigma) )
    
    f.write('\end{tabular} \n'+
            '\end{center}\n')
    f.close()
    
    #print output
    f = open(fname, "r")
    print f.read()
    f.close()

In [61]:
reg1 = pd.read_csv('data/clean_milk1.csv')
print reg1.columns

#variables names
lmilk = ['LWW']
auct_key = ['YEAR','MONTH','DAY','SYSTEM','FMOZONE']
lcts = ['LFMO','LGAS','LPOPUL','LQWW','LMEALS']
dummies = ['COOLER','ESC']
fekeys = ['1A', '3', '6', '7', '9']

lags = 4
lagkeys = [l+str(i) for l in ['LWW_min','LWW_max'] for i in range(1,1+lags)]

bid_key = auct_key + ['VENDOR'] + ['COUNTY']
covariates = lcts + dummies + fekeys
hist = ['INC'] + lagkeys

Index([u'Unnamed: 0', u'YEAR', u'MONTH', u'DAY', u'SYSTEM', u'FMOZONE',
       u'VENDOR', u'COUNTY', u'LWW', u'LFMO', u'LGAS', u'LPOPUL', u'LQWW',
       u'LMEALS', u'COOLER', u'ESC', u'1A', u'3', u'6', u'7', u'9', u'INC',
       u'LWW_min1', u'LWW_min2', u'LWW_min3', u'LWW_min4', u'LWW_max1',
       u'LWW_max2', u'LWW_max3', u'LWW_max4'],
      dtype='object')


In [62]:
nice_ww = 'bids (log-log)'
nice_cov = ['(intercept)', 'raw milk', 'gas',
            'population', 'quantity', 'log meals', 'cooler', 'escalated', 
            'zone 1A', 'zone 3','zone 6', 'zone 7', 'zone 9']
nice_lags = [l+str(i) for l in ['min lag ','max lag '] for i in range(1,1+lags)]
nice_lags = ['incumbency'] + nice_lags

In [63]:
est0 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),1)
write_table('results/ols_results.tex', est0, labels=(nice_ww, nice_cov))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & bids (log-log) & \textbf{  R-squared: } &  0.185 \\ 
\textbf{No. Observations:} & 3662 & \textbf{ AIC:} & -8112.3 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{History 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{(intercept)} & -2.26985 & (0.07203) & -31.51394 & 0.0 \\ \\ 
\textbf{raw milk} & 0.21025 & (0.02618) & 8.03006 & 0.0 \\ \\ 
\textbf{gas} & 0.00915 & (0.00409) & 2.23351 & 0.02558 \\ \\ 
\textbf{population} & 0.02548 & (0.00315) & 8.0969 & 0.0 \\ \\ 
\textbf{quantity} & -0.00017 & (0.00204) & -0.08106 & 0.9354 \\ \\ 
\textbf{log meals} & -0.02267 & (0.00307) & -7.39402 & 0.0 \\ \\ 
\textbf{cooler} & 0.01783 & (0.00296) & 6.02454 & 0.0 \\ \\ 
\textbf{escalated} & -0.01983 & (0.0028) & -7.09227 & 0.0 \\ \\ 
\textbf{zone 1A} & -0.02461 & (0.01894) & -1.29922 & 0.19395 \\ \\ 
\textbf{zone 3} & -0.06472 & (0.00421) & -15.38869 & 0.0 \

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [64]:
est1 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates + hist]),1)
write_table('results/hist_results.tex', est1, labels=(nice_ww, nice_cov + nice_lags))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & bids (log-log) & \textbf{  R-squared: } &  0.212 \\ 
\textbf{No. Observations:} & 3662 & \textbf{ AIC:} & -8220.8 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{History 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{(intercept)} & -1.60081 & (0.10231) & -15.64598 & 0.0 \\ \\ 
\textbf{raw milk} & 0.15289 & (0.02648) & 5.77315 & 0.0 \\ \\ 
\textbf{gas} & 0.01008 & (0.00406) & 2.48672 & 0.01294 \\ \\ 
\textbf{population} & 0.02063 & (0.00314) & 6.56085 & 0.0 \\ \\ 
\textbf{quantity} & 0.00153 & (0.00204) & 0.74757 & 0.45477 \\ \\ 
\textbf{log meals} & -0.0204 & (0.00304) & -6.71197 & 0.0 \\ \\ 
\textbf{cooler} & 0.01853 & (0.00294) & 6.30586 & 0.0 \\ \\ 
\textbf{escalated} & -0.01903 & (0.00275) & -6.9094 & 0.0 \\ \\ 
\textbf{zone 1A} & -0.01474 & (0.01868) & -0.78888 & 0.43023 \\ \\ 
\textbf{zone 3} & -0.06577 & (0.00415) & -15.84827 & 0.0 \\

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [65]:
est2 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),2)
write_table('results/prelim_results.tex', est2, labels=(nice_ww, nice_cov))

\begin{center}  
\begin{tabular}{lclc} 
\toprule 
\textbf{Dep. Variable:} & bids (log-log) & \textbf{  R-squared: } &  0.439 \\ 
\textbf{No. Observations:} & 3662 & \textbf{ AIC:} & -8736.6 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\toprule 
\textbf{History 1} & \textbf{est} & \textbf{std err} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\bottomrule \\ 
\textbf{lambda} & 0.68168  & & & \\ \\ 
\textbf{(intercept)} & -2.14174 & (0.08361) & -25.61663 & 0.0 \\ \\ 
\textbf{raw milk} & 0.19338 & (0.03018) & 6.40793 & 0.0 \\ \\ 
\textbf{gas} & -0.00929 & (0.00472) & -1.96956 & 0.04896 \\ \\ 
\textbf{population} & 0.03934 & (0.00406) & 9.70019 & 0.0 \\ \\ 
\textbf{quantity} & 0.00099 & (0.00229) & 0.43343 & 0.66473 \\ \\ 
\textbf{log meals} & -0.03511 & (0.00394) & -8.92208 & 0.0 \\ \\ 
\textbf{cooler} & 0.01842 & (0.00342) & 5.38549 & 0.0 \\ \\ 
\textbf{escalated} & -0.01938 & (0.00329) & -5.8901 & 0.0 \\ \\ 
\textbf{zone 1A} & -0.04009 & (0.02406) & -1.66597 & 0.09581 \\ \\ 
\textbf{zone 3} 

In [66]:
test1 = estimate(Y, sm.add_constant(X), 2)
test2 = estimate(Y, sm.add_constant(X), 1)

def nonnested_test(model1,model2):
    """test for non nested models quang vuong"""
    
    params1, se1, r21, y1, x1, ncomp1, ll1 = model1
    params2, se2, r22, y2, x2, ncomp2, ll2 = model2
    nobs, k = x1.shape
    
    k1 = params1.shape[1]*ncomp1 - 1 
    k2 = params2.shape[1]*ncomp2 - 1
    
    var1 = (ll1 -ll2).std()
    test1 = (ll1.sum() - ll2.sum() - k1 + k2)*nobs**(-.5)
    test1 = test1/var1
    p1 = 1 - stats.t.cdf(np.abs(test1),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test1),df=(nobs-k1-k2))
    
    var2 =  ((ll1 - ll2)**2).mean()**.5
    test2 = (ll1.sum() - ll2.sum() - k1 + k2 )*nobs**(-.5)
    test2 = test2/var2
    p2 = 1 - stats.t.cdf(np.abs(test2),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test2),df=(nobs-k1-k2))
    
    return test1, test2, p1, p2

print nonnested_test(est2,est1)

(8.86039146307833, 8.762589177157354, 6.13732050035835e-19, 1.4420111216904853e-18)


In [68]:
def write_nonnested(model1,model2,fname):
    test1, test2, p1, p2 = nonnested_test(model1,model2)
    test1, test2, p1, p2 = np.round(test1,5), np.round(test2,5), np.round(p1,5), np.round(p2,5) 
    f = open(fname, "w+")
    f.write('\\begin{center} \n\\begin{tabular}{lcc}')
    f.write('\n\\toprule \n & \\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\')
    f.write('\n\\bottomrule')
    f.write('\n\\textbf{Test 1} & %s & %s \\\\'%(test1,p1))
    f.write('\n\\textbf{Test 2} & %s & %s \\\\'%(test2,p2))
    f.write('\n\\bottomrule \n\\end{tabular}\n\\end{center}')
    f.close()

write_nonnested(est2,est1,'results/test_stat.tex')